# Mask Use Detection - Edge Cases

In [1]:
import numpy as np
import pandas as pd

from skimage.io import imread, imshow

from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier

import data_utils
import ml_utils as ml

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

pd.set_option('display.max_colwidth', 2000)

## Preparing Data

In [2]:
data_path = "data/Mask_DB/"

We will recreate the model that achieved the best results and then test it on images of the class "mask_incorrect_use" that contains images where a mask is present but is not used properly.

### Importing training data

In [3]:
rgb_generator = data_utils.read_images(data_path, class_ignore=['mask_incorrect_use'])
data_hog = data_utils.extract_features(rgb_generator, 'hog')
data_hog = data_hog.rename(columns={data_hog.columns[-1]: "class"})

In [4]:
data_hog = data_utils.train_test_val_split(data_hog, 0.6, 0.2)
X_train = data_hog[0][data_hog[0].columns[:-1]]
y_train = data_hog[0]['class']

In [5]:
y_train

1331    1.0
1746    1.0
326     0.0
560     0.0
1894    1.0
       ... 
1638    1.0
1095    1.0
1130    1.0
1294    1.0
860     0.0
Name: class, Length: 1252, dtype: float64

### Importing incorrect mask use data

In [6]:
rgb_generator = data_utils.read_images(data_path, class_ignore=['with_mask', 'without_mask'])
data_hog = data_utils.extract_features(rgb_generator, 'hog')
data_hog = data_hog.drop(data_hog.columns[-1], axis='columns')

In [7]:
X_test = data_hog
y_test = pd.Series(np.ones((X_test.shape[0],)))

### Feature Scaling

In [8]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

### Dimensionality Reduction

In [9]:
X_train, X_test = ml.reduce_dimensions(X_train, y_train, X_test, 'pca', 27)

## Model Training

In [10]:
svm = SVC(kernel='rbf', C=1000, gamma=0.01, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(32,), activation='tanh', alpha=0.5, learning_rate_init=0.01, max_iter=1000, random_state=42)
xgb = XGBClassifier(n_estimators=1000, max_depth=8, learning_rate=0.01, subsample=0.5, random_state=42, n_jobs=-1)
rf  = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, random_state=42)

level_0 = [
    ('mlp', mlp),
    ('svm', svm),
    ('xgb', xgb),
    ('rf ' , rf)
]

stacking = StackingClassifier(estimators=level_0, final_estimator=svm, passthrough=True)
stacking.fit(X_train, y_train)

StackingClassifier(estimators=[('mlp',
                                MLPClassifier(activation='tanh', alpha=0.5,
                                              hidden_layer_sizes=(32,),
                                              learning_rate_init=0.01,
                                              max_iter=1000, random_state=42)),
                               ('svm',
                                SVC(C=1000, gamma=0.01, random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stop...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=8,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=1000, n_jobs=-1,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=42, ...)),
                               ('rf ',
                                RandomForestClassifier(min_samples_leaf=2,
                                                       random_state=42))],
                   final_estimator=SVC(C=1000, gamma=0.01, random_state=42),
                   passthrough=True)

## Testing

In [11]:
y_pred = stacking.predict(X_test)

In [12]:
print("Images classified correctly:", (y_pred == y_test).sum())
print("Accuracy:", (y_pred == y_test).sum() / y_test.shape[0])

Images classified correctly: 11
Accuracy: 0.19642857142857142


The accuracy seems to be very low. This possibly happened because masks were not present in the entirety of the "without_mask" class, and the algorithms learned to strongly corelate this feature with the class.